<b>Dataset</b> - Kaggle Dataset of flight delays for the year 2015 as it has both categorical and numerical features. With approximately 5 million rows, this dataset will be good for judging the performance in terms of both speed and accuracy of tuned models for each type of boosting. To avoid long waiting, I will be using a 1% subset of this data ~ 50k rows.

The features used for modeling:
- <b>MONTH, DAY, DAY_OF_WEEK:</b> data type int
- <b>AIRLINE and FLIGHT_NUMBER:</b> data type int
- <b>ORIGIN_AIRPORT and DESTINATION_AIRPORT:</b> data type string
- <b>DEPARTURE_TIME:</b> data type float
- <b>ARRIVAL_DELAY:</b> this will be the target and is transformed into boolean variable indicating delay of more than 10 minutes
- <b>DISTANCE and AIR_TIME:</b> data type float

### Data Preparation

In [1]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split

data = pd.read_csv("flights.csv")
data = data.sample(frac = 0.01, random_state=10)

data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT", 
             "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]

data.dropna(inplace=True)

data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1

cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1
 
train, test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"],
                                                random_state=10, test_size=0.25)


C:\Users\Kerry\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### XGBoost

In [2]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]), 
            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

from datetime import datetime 

# Parameter Tuning
start = datetime.now()

model = xgb.XGBClassifier()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}

grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, verbose=10, n_jobs=-1)
grid_search.fit(train, y_train)

stop = datetime.now()

grid_search.best_estimator_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 17.3min finished


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=6, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [3]:
from datetime import datetime 
start1 = datetime.now()
model = xgb.XGBClassifier(max_depth=10, min_child_weight=6,  n_estimators=200,\
                          n_jobs=1, verbose=1, learning_rate=0.05)
model.fit(train,y_train)
stop1 = datetime.now()

In [4]:
auc(model, train, test)

(0.9256025656508435, 0.7125663850538142)

In [5]:
execution_time_xgb = stop-start 
execution_time_xgb

datetime.timedelta(seconds=1052, microseconds=414018)

In [6]:
execution_time_xgb1 = stop1-start1 
execution_time_xgb1

datetime.timedelta(seconds=13, microseconds=85029)

### LightGBM

In [7]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
            metrics.roc_auc_score(y_test,m.predict(test)))

from datetime import datetime 

# Parameter Tuning
start2 = datetime.now()
lg = lgb.LGBMClassifier(silent=False)
param_dists = {"max_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dists, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(train,y_train)

stop2 = datetime.now()

grid_search.best_estimator_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  8.3min finished


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.05, max_depth=25,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=200, n_jobs=-1, num_leaves=300, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [8]:
execution_time_lgb = stop2-start2
execution_time_lgb

datetime.timedelta(seconds=502, microseconds=858548)

In [9]:
from datetime import datetime 
start3 = datetime.now()
d_train = lgb.Dataset(train, label=y_train, free_raw_data=False)
params = {"max_depth": 25, "learning_rate" : 0.05, "num_leaves": 300,  "n_estimators": 200}

# Without Categorical Features
model2 = lgb.train(params, d_train)
stop3 = datetime.now()
auc2(model2, train, test)

C:\Users\Kerry\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


(0.9890966235699548, 0.7049697275626666)

In [10]:
execution_time_lgb = stop3-start3
execution_time_lgb

datetime.timedelta(seconds=4, microseconds=694948)

In [11]:
#With Catgeorical Features
from datetime import datetime 
start4 = datetime.now()
cate_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT"]
model3 = lgb.train(params, d_train, categorical_feature = cate_features_name)
stop4 = datetime.now()
auc2(model3, train, test)

C:\Users\Kerry\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Kerry\Anaconda3\lib\site-packages\lightgbm\basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['AIRLINE', 'DAY', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT', 'MONTH', 'ORIGIN_AIRPORT']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


(0.9997287165610917, 0.6927970559212121)

In [12]:
execution_time_lgb = stop4-start4
execution_time_lgb

datetime.timedelta(seconds=6, microseconds=457356)

In [13]:
import catboost as cb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

cat_features_index = [0,1,2,3,4,5,6]

def auc3(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

from datetime import datetime 
start5 = datetime.now()

cab = cb.CatBoostClassifier()
params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
grid_search = GridSearchCV(cab, param_grid=params, scoring="roc_auc", cv = 3)
grid_search.fit(train, y_train)

stop5 = datetime.now()

grid_search.cv_results_, grid_search.best_params_, grid_search.best_score_

0:	learn: 0.6809571	total: 285ms	remaining: 1m 25s
1:	learn: 0.6699237	total: 297ms	remaining: 44.2s
2:	learn: 0.6594560	total: 324ms	remaining: 32.1s
3:	learn: 0.6496294	total: 353ms	remaining: 26.1s
4:	learn: 0.6404491	total: 365ms	remaining: 21.5s
5:	learn: 0.6319287	total: 387ms	remaining: 19s
6:	learn: 0.6247064	total: 411ms	remaining: 17.2s
7:	learn: 0.6169569	total: 423ms	remaining: 15.4s
8:	learn: 0.6098048	total: 435ms	remaining: 14.1s
9:	learn: 0.6031784	total: 452ms	remaining: 13.1s
10:	learn: 0.5967758	total: 550ms	remaining: 14.5s
11:	learn: 0.5908704	total: 563ms	remaining: 13.5s
12:	learn: 0.5854861	total: 608ms	remaining: 13.4s
13:	learn: 0.5802278	total: 618ms	remaining: 12.6s
14:	learn: 0.5753227	total: 629ms	remaining: 12s
15:	learn: 0.5709531	total: 641ms	remaining: 11.4s
16:	learn: 0.5665448	total: 657ms	remaining: 10.9s
17:	learn: 0.5625716	total: 677ms	remaining: 10.6s
18:	learn: 0.5589610	total: 689ms	remaining: 10.2s
19:	learn: 0.5554187	total: 699ms	remaining:

162:	learn: 0.4875035	total: 2.75s	remaining: 2.31s
163:	learn: 0.4874546	total: 2.76s	remaining: 2.29s
164:	learn: 0.4873497	total: 2.77s	remaining: 2.27s
165:	learn: 0.4872823	total: 2.79s	remaining: 2.25s
166:	learn: 0.4871863	total: 2.82s	remaining: 2.25s
167:	learn: 0.4870487	total: 2.84s	remaining: 2.23s
168:	learn: 0.4869720	total: 2.85s	remaining: 2.21s
169:	learn: 0.4869243	total: 2.88s	remaining: 2.2s
170:	learn: 0.4868774	total: 2.89s	remaining: 2.18s
171:	learn: 0.4868319	total: 2.9s	remaining: 2.16s
172:	learn: 0.4867959	total: 2.92s	remaining: 2.14s
173:	learn: 0.4866959	total: 2.93s	remaining: 2.12s
174:	learn: 0.4866179	total: 2.94s	remaining: 2.1s
175:	learn: 0.4865641	total: 2.95s	remaining: 2.08s
176:	learn: 0.4864335	total: 2.97s	remaining: 2.06s
177:	learn: 0.4863707	total: 2.99s	remaining: 2.05s
178:	learn: 0.4863078	total: 3.02s	remaining: 2.04s
179:	learn: 0.4862351	total: 3.05s	remaining: 2.03s
180:	learn: 0.4861941	total: 3.06s	remaining: 2.02s
181:	learn: 0.4

29:	learn: 0.5330586	total: 402ms	remaining: 3.62s
30:	learn: 0.5313367	total: 433ms	remaining: 3.76s
31:	learn: 0.5296598	total: 469ms	remaining: 3.93s
32:	learn: 0.5280906	total: 494ms	remaining: 3.99s
33:	learn: 0.5268831	total: 504ms	remaining: 3.94s
34:	learn: 0.5257026	total: 517ms	remaining: 3.92s
35:	learn: 0.5245873	total: 529ms	remaining: 3.88s
36:	learn: 0.5233796	total: 546ms	remaining: 3.88s
37:	learn: 0.5221957	total: 558ms	remaining: 3.84s
38:	learn: 0.5208518	total: 571ms	remaining: 3.82s
39:	learn: 0.5199855	total: 591ms	remaining: 3.84s
40:	learn: 0.5189115	total: 602ms	remaining: 3.81s
41:	learn: 0.5178368	total: 612ms	remaining: 3.76s
42:	learn: 0.5170340	total: 627ms	remaining: 3.75s
43:	learn: 0.5159958	total: 637ms	remaining: 3.71s
44:	learn: 0.5151517	total: 647ms	remaining: 3.67s
45:	learn: 0.5144530	total: 658ms	remaining: 3.63s
46:	learn: 0.5136015	total: 668ms	remaining: 3.6s
47:	learn: 0.5128011	total: 679ms	remaining: 3.56s
48:	learn: 0.5121832	total: 689m

190:	learn: 0.4882920	total: 2.59s	remaining: 1.48s
191:	learn: 0.4882189	total: 2.6s	remaining: 1.46s
192:	learn: 0.4881377	total: 2.61s	remaining: 1.45s
193:	learn: 0.4880444	total: 2.62s	remaining: 1.43s
194:	learn: 0.4879888	total: 2.63s	remaining: 1.42s
195:	learn: 0.4878896	total: 2.64s	remaining: 1.4s
196:	learn: 0.4878364	total: 2.65s	remaining: 1.39s
197:	learn: 0.4877766	total: 2.66s	remaining: 1.37s
198:	learn: 0.4877312	total: 2.67s	remaining: 1.36s
199:	learn: 0.4876686	total: 2.69s	remaining: 1.34s
200:	learn: 0.4875788	total: 2.7s	remaining: 1.33s
201:	learn: 0.4875372	total: 2.71s	remaining: 1.31s
202:	learn: 0.4874122	total: 2.71s	remaining: 1.3s
203:	learn: 0.4873422	total: 2.72s	remaining: 1.28s
204:	learn: 0.4872888	total: 2.73s	remaining: 1.27s
205:	learn: 0.4871921	total: 2.74s	remaining: 1.25s
206:	learn: 0.4870742	total: 2.75s	remaining: 1.24s
207:	learn: 0.4870419	total: 2.76s	remaining: 1.22s
208:	learn: 0.4869846	total: 2.77s	remaining: 1.21s
209:	learn: 0.48

57:	learn: 0.5051841	total: 528ms	remaining: 2.2s
58:	learn: 0.5047131	total: 537ms	remaining: 2.19s
59:	learn: 0.5042295	total: 547ms	remaining: 2.19s
60:	learn: 0.5037699	total: 556ms	remaining: 2.18s
61:	learn: 0.5033907	total: 564ms	remaining: 2.17s
62:	learn: 0.5030330	total: 573ms	remaining: 2.16s
63:	learn: 0.5027530	total: 582ms	remaining: 2.15s
64:	learn: 0.5025074	total: 591ms	remaining: 2.13s
65:	learn: 0.5021825	total: 600ms	remaining: 2.13s
66:	learn: 0.5018459	total: 608ms	remaining: 2.11s
67:	learn: 0.5015289	total: 617ms	remaining: 2.11s
68:	learn: 0.5012429	total: 628ms	remaining: 2.1s
69:	learn: 0.5009537	total: 637ms	remaining: 2.09s
70:	learn: 0.5006507	total: 647ms	remaining: 2.09s
71:	learn: 0.5004038	total: 658ms	remaining: 2.08s
72:	learn: 0.5001183	total: 666ms	remaining: 2.07s
73:	learn: 0.4999042	total: 675ms	remaining: 2.06s
74:	learn: 0.4996824	total: 687ms	remaining: 2.06s
75:	learn: 0.4994696	total: 697ms	remaining: 2.06s
76:	learn: 0.4992037	total: 712ms

...

280:	learn: 0.2597296	total: 30s	remaining: 2.03s
281:	learn: 0.2593410	total: 30.1s	remaining: 1.92s
282:	learn: 0.2588152	total: 30.2s	remaining: 1.82s
283:	learn: 0.2584002	total: 30.4s	remaining: 1.71s
284:	learn: 0.2580339	total: 30.5s	remaining: 1.6s
285:	learn: 0.2574911	total: 30.6s	remaining: 1.5s
286:	learn: 0.2566613	total: 30.7s	remaining: 1.39s
287:	learn: 0.2562236	total: 30.8s	remaining: 1.28s
288:	learn: 0.2556581	total: 30.9s	remaining: 1.18s
289:	learn: 0.2550554	total: 31s	remaining: 1.07s
290:	learn: 0.2546669	total: 31.2s	remaining: 964ms
291:	learn: 0.2543721	total: 31.3s	remaining: 857ms
292:	learn: 0.2537472	total: 31.4s	remaining: 750ms
293:	learn: 0.2534545	total: 31.5s	remaining: 643ms
294:	learn: 0.2531253	total: 31.6s	remaining: 536ms
295:	learn: 0.2526242	total: 31.7s	remaining: 429ms
296:	learn: 0.2521726	total: 31.8s	remaining: 322ms
297:	learn: 0.2516284	total: 31.9s	remaining: 214ms
298:	learn: 0.2511044	total: 32s	remaining: 107ms
299:	learn: 0.250850

141:	learn: 0.3409215	total: 15.2s	remaining: 16.9s
142:	learn: 0.3402386	total: 15.3s	remaining: 16.8s
143:	learn: 0.3397760	total: 15.4s	remaining: 16.7s
144:	learn: 0.3388022	total: 15.5s	remaining: 16.6s
145:	learn: 0.3378238	total: 15.6s	remaining: 16.5s
146:	learn: 0.3366466	total: 15.8s	remaining: 16.4s
147:	learn: 0.3351346	total: 15.9s	remaining: 16.3s
148:	learn: 0.3347432	total: 16s	remaining: 16.2s
149:	learn: 0.3340144	total: 16.2s	remaining: 16.2s
150:	learn: 0.3332489	total: 16.3s	remaining: 16.1s
151:	learn: 0.3328729	total: 16.4s	remaining: 16s
152:	learn: 0.3316637	total: 16.5s	remaining: 15.9s
153:	learn: 0.3311891	total: 16.6s	remaining: 15.8s
154:	learn: 0.3306050	total: 16.8s	remaining: 15.7s
155:	learn: 0.3298397	total: 16.9s	remaining: 15.6s
156:	learn: 0.3289936	total: 17s	remaining: 15.5s
157:	learn: 0.3284516	total: 17.1s	remaining: 15.4s
158:	learn: 0.3279256	total: 17.2s	remaining: 15.3s
159:	learn: 0.3274944	total: 17.3s	remaining: 15.2s
160:	learn: 0.3265

0:	learn: 0.6372427	total: 15.6ms	remaining: 4.66s
1:	learn: 0.5987985	total: 35ms	remaining: 5.21s
2:	learn: 0.5712184	total: 48.7ms	remaining: 4.82s
3:	learn: 0.5518582	total: 65.6ms	remaining: 4.86s
4:	learn: 0.5381022	total: 81.7ms	remaining: 4.82s
5:	learn: 0.5282065	total: 97.4ms	remaining: 4.77s
6:	learn: 0.5210165	total: 114ms	remaining: 4.75s
7:	learn: 0.5151370	total: 130ms	remaining: 4.75s
8:	learn: 0.5114281	total: 146ms	remaining: 4.73s
9:	learn: 0.5082715	total: 163ms	remaining: 4.72s
10:	learn: 0.5054974	total: 177ms	remaining: 4.65s
11:	learn: 0.5039080	total: 194ms	remaining: 4.66s
12:	learn: 0.5023021	total: 211ms	remaining: 4.66s
13:	learn: 0.5008525	total: 231ms	remaining: 4.71s
14:	learn: 0.4994765	total: 244ms	remaining: 4.64s
15:	learn: 0.4986077	total: 260ms	remaining: 4.61s
16:	learn: 0.4975655	total: 276ms	remaining: 4.6s
17:	learn: 0.4968286	total: 290ms	remaining: 4.55s
18:	learn: 0.4961508	total: 310ms	remaining: 4.58s
19:	learn: 0.4954372	total: 324ms	rema

161:	learn: 0.4617887	total: 3.66s	remaining: 3.12s
162:	learn: 0.4616568	total: 3.69s	remaining: 3.1s
163:	learn: 0.4614862	total: 3.72s	remaining: 3.09s
164:	learn: 0.4614075	total: 3.74s	remaining: 3.06s
165:	learn: 0.4612662	total: 3.77s	remaining: 3.05s
166:	learn: 0.4610916	total: 3.81s	remaining: 3.03s
167:	learn: 0.4609350	total: 3.83s	remaining: 3.01s
168:	learn: 0.4608192	total: 3.85s	remaining: 2.99s
169:	learn: 0.4607077	total: 3.88s	remaining: 2.97s
170:	learn: 0.4606132	total: 3.9s	remaining: 2.94s
171:	learn: 0.4604895	total: 3.93s	remaining: 2.92s
172:	learn: 0.4603945	total: 3.95s	remaining: 2.9s
173:	learn: 0.4602799	total: 3.97s	remaining: 2.88s
174:	learn: 0.4601406	total: 3.99s	remaining: 2.85s
175:	learn: 0.4600253	total: 4.02s	remaining: 2.83s
176:	learn: 0.4599177	total: 4.04s	remaining: 2.8s
177:	learn: 0.4597758	total: 4.06s	remaining: 2.78s
178:	learn: 0.4596723	total: 4.08s	remaining: 2.76s
179:	learn: 0.4595649	total: 4.1s	remaining: 2.73s
180:	learn: 0.459

({'mean_fit_time': array([ 5.14184237,  4.11287681,  4.05300983,  4.06798553,  4.25925906,
          4.18831881,  4.17663678,  4.27758304,  4.26951996,  6.89187574,
          8.40593855,  7.85819411,  7.68170079,  8.01185091,  7.76588074,
          7.91951124,  7.39373223,  7.76867135, 33.20755959, 33.87388031,
         33.01634677, 32.85077763, 32.47248507, 32.83023119, 32.41059589,
         33.28668523, 32.70527204]),
  'std_fit_time': array([1.47807819, 0.15261282, 0.15250159, 0.14456792, 0.15161358,
         0.24142621, 0.31099691, 0.3128639 , 0.24661151, 0.35275908,
         1.14051156, 0.37609015, 0.24647699, 0.2162501 , 0.09573827,
         0.10986545, 0.65119927, 0.32792796, 1.61374785, 0.92133869,
         0.37365941, 0.70982239, 0.00739143, 0.29701327, 0.04753457,
         0.41372454, 0.13022788]),
  'mean_score_time': array([0.05171013, 0.01201463, 0.01163983, 0.01201121, 0.01067877,
         0.0146517 , 0.01201741, 0.01183915, 0.01067821, 0.01199706,
         0.01067313, 0.

In [14]:
execution_time_cb = stop5-start5
execution_time_cb

datetime.timedelta(seconds=1224, microseconds=54158)

In [16]:
# Without Categorical features
from datetime import datetime 
start6 = datetime.now()

clf = cb.CatBoostClassifier(eval_metric="AUC", depth=4, iterations= 300, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train)

stop6 = datetime.now()

auc3(clf, train, test)

0:	total: 11.7ms	remaining: 3.51s
1:	total: 24.8ms	remaining: 3.69s
2:	total: 36.2ms	remaining: 3.58s
3:	total: 49.1ms	remaining: 3.63s
4:	total: 60.8ms	remaining: 3.58s
5:	total: 72.9ms	remaining: 3.57s
6:	total: 84.2ms	remaining: 3.52s
7:	total: 101ms	remaining: 3.69s
8:	total: 112ms	remaining: 3.63s
9:	total: 125ms	remaining: 3.63s
10:	total: 136ms	remaining: 3.58s
11:	total: 149ms	remaining: 3.58s
12:	total: 162ms	remaining: 3.57s
13:	total: 175ms	remaining: 3.57s
14:	total: 186ms	remaining: 3.53s
15:	total: 199ms	remaining: 3.53s
16:	total: 218ms	remaining: 3.62s
17:	total: 231ms	remaining: 3.62s
18:	total: 245ms	remaining: 3.62s
19:	total: 257ms	remaining: 3.59s
20:	total: 270ms	remaining: 3.58s
21:	total: 283ms	remaining: 3.57s
22:	total: 295ms	remaining: 3.55s
23:	total: 308ms	remaining: 3.54s
24:	total: 319ms	remaining: 3.51s
25:	total: 331ms	remaining: 3.48s
26:	total: 343ms	remaining: 3.47s
27:	total: 356ms	remaining: 3.46s
28:	total: 369ms	remaining: 3.45s
29:	total: 382ms	

241:	total: 4.51s	remaining: 1.08s
242:	total: 4.53s	remaining: 1.06s
243:	total: 4.55s	remaining: 1.04s
244:	total: 4.56s	remaining: 1.02s
245:	total: 4.58s	remaining: 1s
246:	total: 4.6s	remaining: 987ms
247:	total: 4.62s	remaining: 968ms
248:	total: 4.63s	remaining: 949ms
249:	total: 4.64s	remaining: 929ms
250:	total: 4.66s	remaining: 910ms
251:	total: 4.67s	remaining: 890ms
252:	total: 4.69s	remaining: 872ms
253:	total: 4.71s	remaining: 852ms
254:	total: 4.72s	remaining: 832ms
255:	total: 4.73s	remaining: 813ms
256:	total: 4.74s	remaining: 794ms
257:	total: 4.76s	remaining: 776ms
258:	total: 4.79s	remaining: 758ms
259:	total: 4.8s	remaining: 739ms
260:	total: 4.82s	remaining: 720ms
261:	total: 4.83s	remaining: 701ms
262:	total: 4.85s	remaining: 682ms
263:	total: 4.86s	remaining: 663ms
264:	total: 4.88s	remaining: 645ms
265:	total: 4.9s	remaining: 627ms
266:	total: 4.92s	remaining: 608ms
267:	total: 4.93s	remaining: 589ms
268:	total: 4.94s	remaining: 570ms
269:	total: 4.96s	remainin

(0.7581917055605452, 0.7111814854434477)

In [17]:
execution_time_cb = stop6-start6
execution_time_cb

datetime.timedelta(seconds=5, microseconds=868052)

In [18]:
# With Categorical features
from datetime import datetime 
start7 = datetime.now()

clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
                            depth=4, iterations= 300, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train, cat_features= cat_features_index)

stop7 = datetime.now()

auc3(clf, train, test)

0:	total: 91.1ms	remaining: 27.2s
1:	total: 145ms	remaining: 21.6s
2:	total: 227ms	remaining: 22.5s
3:	total: 267ms	remaining: 19.7s
4:	total: 334ms	remaining: 19.7s
5:	total: 373ms	remaining: 18.3s
6:	total: 413ms	remaining: 17.3s
7:	total: 451ms	remaining: 16.5s
8:	total: 553ms	remaining: 17.9s
9:	total: 633ms	remaining: 18.3s
10:	total: 692ms	remaining: 18.2s
11:	total: 738ms	remaining: 17.7s
12:	total: 798ms	remaining: 17.6s
13:	total: 841ms	remaining: 17.2s
14:	total: 900ms	remaining: 17.1s
15:	total: 960ms	remaining: 17s
16:	total: 1.01s	remaining: 16.8s
17:	total: 1.06s	remaining: 16.6s
18:	total: 1.12s	remaining: 16.5s
19:	total: 1.17s	remaining: 16.4s
20:	total: 1.23s	remaining: 16.3s
21:	total: 1.28s	remaining: 16.1s
22:	total: 1.33s	remaining: 16s
23:	total: 1.38s	remaining: 15.9s
24:	total: 1.43s	remaining: 15.7s
25:	total: 1.49s	remaining: 15.7s
26:	total: 1.56s	remaining: 15.8s
27:	total: 1.61s	remaining: 15.7s
28:	total: 1.68s	remaining: 15.7s
29:	total: 1.74s	remaining:

240:	total: 15.6s	remaining: 3.83s
241:	total: 15.7s	remaining: 3.77s
242:	total: 15.8s	remaining: 3.7s
243:	total: 15.9s	remaining: 3.64s
244:	total: 15.9s	remaining: 3.57s
245:	total: 16s	remaining: 3.51s
246:	total: 16.1s	remaining: 3.44s
247:	total: 16.1s	remaining: 3.38s
248:	total: 16.2s	remaining: 3.32s
249:	total: 16.3s	remaining: 3.25s
250:	total: 16.3s	remaining: 3.18s
251:	total: 16.4s	remaining: 3.12s
252:	total: 16.4s	remaining: 3.05s
253:	total: 16.5s	remaining: 2.99s
254:	total: 16.6s	remaining: 2.92s
255:	total: 16.6s	remaining: 2.86s
256:	total: 16.7s	remaining: 2.79s
257:	total: 16.8s	remaining: 2.73s
258:	total: 16.8s	remaining: 2.66s
259:	total: 16.9s	remaining: 2.6s
260:	total: 16.9s	remaining: 2.53s
261:	total: 17s	remaining: 2.47s
262:	total: 17.1s	remaining: 2.4s
263:	total: 17.2s	remaining: 2.34s
264:	total: 17.2s	remaining: 2.27s
265:	total: 17.3s	remaining: 2.21s
266:	total: 17.4s	remaining: 2.15s
267:	total: 17.5s	remaining: 2.09s
268:	total: 17.6s	remaining

(0.7883548816786108, 0.7127829275766384)

In [19]:
execution_time_cb = stop7-start7
execution_time_cb

datetime.timedelta(seconds=20, microseconds=889372)

1. <b>XGBoost:</b> 
    - GridSearch time: 1052 seconds 
        - Training time: 13 seconds ; Training AUC: 0.9256025656508435; Testing AUC: 0.7125663850538142
<br /><br />
2. <b>LightGBM:</b> 
    - GridSearch time: 502 seconds 
    - Without Categorical features: 
        - Training time: 4 seconds ; Training AUC: 0.9890966235699548 ; Testing AUC: 0.7049697275626666  
    - With Categorical features: 
        - Training time: 6 seconds ; Training AUC: 0.9997287165610917 ; Testing AUC: 0.6927970559212121
<br /><br />
3. <b>Catboost:</b> 
    - GridSearch time: 1224 seconds
    - Without Categorical features: 
        - Training time: 5 seconds ; Training AUC: 0.7581917055605452 ; Testing AUC: 0.7111814854434477 
    - With Categorical features: 
        - Training time: 20 seconds ; Training AUC: 0.7883548816786108 ; Testing AUC: 0.7127829275766384

<b>Scores</b>
<br />
1st: Catboost with Categorical features (Testing Score: 0.7127829275766384)
<br />
2nd: XGBoost (Testing Score: 0.7125663850538142)
<br />
3rd: LightGBM
<br />

<b>Time</b>
<br />
1st: LightGBM
<br />
2nd: XGBoost
<br />
3rd: Catboost
<br />

<b>Overall</b>
<br />
1st: XGBoost